percent_true
has_ergot indicates if there is at least one true
sum_severity
previous_one_has_ergot indicates if there is at least one true last year
previous_one_severity
previous_two_has_ergot indicates if there is at least one true two years ago
previous_two_severity

we also need a car_uid column to link the prov - district in this table to the ag region boundary table

the final ergot table has

In [1]:
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

e:\Programming\CGC_Grain_Outcome_Predictions\env\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\danew\AppData\Local\Temp\ipykernel_24536\3364527155.py:2: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be s

In [2]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [3]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [4]:
query = "SELECT * FROM public.ergot_sample"
ergot = pd.read_sql_query(query, conn)  # type: ignore
#components = pd.DataFrame(query, columns=["soil_id", "poly_id"])  # type: ignore

In [5]:
ergot

,sample_id,year,province,crop_district,incidence,severity
0,1,1995,AB,1,False,0.0
1,2,1995,AB,1,False,0.0
2,3,1995,AB,1,False,0.0
3,4,1995,AB,1,False,0.0
4,5,1995,AB,1,False,0.0
...,...,...,...,...,...,...
158684,158685,2022,SK,81,False,0.0
158685,158686,2022,SK,81,False,0.0
158686,158687,2022,SK,81,False,0.0
158687,158688,2022,AB,7,False,0.0


In [6]:
# calculate the car_uid
ergot.loc[ergot['province'] == 'MB', 'car_uid'] = ergot.loc[ergot['province'] == 'MB', 'crop_district'] + 4600
ergot.loc[ergot['province'] == 'SK', 'car_uid'] = (ergot.loc[ergot['province'] == 'SK', 'crop_district'] - 1) + 4700 
ergot.loc[ergot['province'] == 'AB', 'car_uid'] = (ergot.loc[ergot['province'] == 'AB', 'crop_district'] * 10) + 4800

In [7]:
ergot

,sample_id,year,province,crop_district,incidence,severity,car_uid
0,1,1995,AB,1,False,0.0,4810.0
1,2,1995,AB,1,False,0.0,4810.0
2,3,1995,AB,1,False,0.0,4810.0
3,4,1995,AB,1,False,0.0,4810.0
4,5,1995,AB,1,False,0.0,4810.0
...,...,...,...,...,...,...,...
158684,158685,2022,SK,81,False,0.0,4780.0
158685,158686,2022,SK,81,False,0.0,4780.0
158686,158687,2022,SK,81,False,0.0,4780.0
158687,158688,2022,AB,7,False,0.0,4870.0


In [8]:
ergot[['car_uid']] = ergot[['car_uid']].astype(int)

In [9]:
ergot

,sample_id,year,province,crop_district,incidence,severity,car_uid
0,1,1995,AB,1,False,0.0,4810
1,2,1995,AB,1,False,0.0,4810
2,3,1995,AB,1,False,0.0,4810
3,4,1995,AB,1,False,0.0,4810
4,5,1995,AB,1,False,0.0,4810
...,...,...,...,...,...,...,...
158684,158685,2022,SK,81,False,0.0,4780
158685,158686,2022,SK,81,False,0.0,4780
158686,158687,2022,SK,81,False,0.0,4780
158687,158688,2022,AB,7,False,0.0,4870


In [ ]:
query = sq.text("select car_uid, geometry FROM public.census_ag_regions")  # type: ignore
agRegions = gpd.GeoDataFrame.from_postgis(query, conn, crs="EPSG:3347", geom_col="geometry")  # type: ignore

In [ ]:
touches = {}

for index1, agRegion1 in agRegions.iterrows():
    currTouches = []
    
    for index2, agRegion2 in agRegions.iterrows():
        if(agRegion1['geometry'].touches(agRegion2['geometry'])):
            currTouches.append(agRegion2['car_uid'])

    touches[str(agRegion1['car_uid'])] = currTouches

In [10]:
ergot['percnt_true'] = None
ergot['has_ergot'] = False
ergot['sumSeverity'] = 0
ergot['present_prev1'] = False
ergot['present_prev2'] = False
ergot['present_prev3'] = False
ergot['present_in_neighbor'] = False

In [30]:
for year in ergot['year'].unique():
    for uid in ergot['car_uid'].unique():
        relSamples = ergot.query(f'year == {year} and car_uid == {uid}')
        neighborSamples = ergot.query(f'year == {year} and car_uid in {touches[str(uid)]}')
        
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'percnt_true'] = relSamples['incidence'].sum()/len(relSamples.index)
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'has_ergot'] = relSamples['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'sumSeverity'] = relSamples['severity'].sum()

        prev1Year = ergot.query(f'year == {year - 1} and car_uid == {uid}')
        prev2Year = ergot.query(f'year == {year - 2} and car_uid == {uid}')
        prev3Year = ergot.query(f'year == {year - 3} and car_uid == {uid}')
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev1'] = prev1Year['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev2'] = prev2Year['incidence'].sum() > 0
        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_prev3'] = prev3Year['incidence'].sum() > 0

        ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'present_in_neighbor'] = neighborSamples['incidence'].sum() > 0

C:\Users\danew\AppData\Local\Temp\ipykernel_24536\1190076422.py:6: RuntimeWarning: invalid value encountered in scalar divide
  ergot.loc[(ergot['year'] == year) & (ergot['car_uid'] == uid), 'percnt_true'] = relSamples['incidence'].sum()/len(relSamples.index)


In [31]:
ergot

,sample_id,year,province,crop_district,incidence,severity,car_uid,percnt_true,has_ergot,sumSeverity,present_prev1,present_prev2,present_prev3,present_in_neighbor
0,1,1995,AB,1,False,0.0,4810,0.0,False,0.00,False,False,False,False
1,2,1995,AB,1,False,0.0,4810,0.0,False,0.00,False,False,False,False
2,3,1995,AB,1,False,0.0,4810,0.0,False,0.00,False,False,False,False
3,4,1995,AB,1,False,0.0,4810,0.0,False,0.00,False,False,False,False
4,5,1995,AB,1,False,0.0,4810,0.0,False,0.00,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158684,158685,2022,SK,81,False,0.0,4780,0.244318,True,0.51,True,True,True,True
158685,158686,2022,SK,81,False,0.0,4780,0.244318,True,0.51,True,True,True,True
158686,158687,2022,SK,81,False,0.0,4780,0.244318,True,0.51,True,True,True,True
158687,158688,2022,AB,7,False,0.0,4870,0.180723,True,0.30,True,True,True,True


In [18]:
db.cleanup()